# Analyze the effect of classification on the speed data

<b>Intuition :</b> Literature in speed detection suggests that usage of hybrid models increases precision of the model. But these classifications are made strictly between slower and faster speeds or walking and runnning speeds. So we first test the effectiveness of classifier to split the speeds between slower and higher speeds. Then we try to see if there is any improvement in the precision in speed detection using the classifier with the highest accuracy. NOte: The regression algorithm used here will be RandomForest as the best regression model was found to be random forests. 

In [ ]:
## Algorithms 